### Build a Simple LLM Application with LCEL.

This application will translate Text from English into another language.

This is a relatively simple LLM application - it's just a single LLM call plus some prompting. 

### Open AI API Key and 
### Open Source models--Llama3(Meta), Gemma2(Google), mistral--Groq

In [20]:

import os
from dotenv import load_dotenv
load_dotenv()


True

In [21]:
import openai
openai.api_key= os.getenv("OPENAI_API_KEY")

groq_api_key= os.getenv("GROQ_API_KEY")

In [22]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI


In [23]:
model = ChatGroq(model="gemma2-9b-it", groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002AEA3514800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002AEA44FED80>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage

messages= [
        SystemMessage(content= "Translate the following from English to French"),
        HumanMessage(content= "Hello How are you?")
]

In [25]:
result= model.invoke(messages)
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal option, used with people you don\'t know well or in professional settings.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (Used with friends and family.)\n* **Coucou, ça va ?** (A very casual way to say hello, similar to "Hey, how\'s it going?")\n\n\nLet me know if you\'d like more translations or have other phrases you want to learn!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 21, 'total_tokens': 149, 'completion_time': 0.232727273, 'prompt_time': 0.00132518, 'queue_time': 0.25475564, 'total_time': 0.234052453}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7821c887-0319-4c7c-bbb5-2c84e8f4dbea-0', usage_metadata={'input_tokens': 21, 'output_tokens': 128, 'to

In [26]:
from langchain_core.output_parsers import StrOutputParser

parser= StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal option, used with people you don\'t know well or in professional settings.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (Used with friends and family.)\n* **Coucou, ça va ?** (A very casual way to say hello, similar to "Hey, how\'s it going?")\n\n\nLet me know if you\'d like more translations or have other phrases you want to learn!\n'

### Using LCEL(LangChain Expression Language)-->> Chain the Components

In [27]:
## This is LCEL

chain= model|parser

chain.invoke(messages)

'Here are a couple of ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment ça va ?** (This is a more casual way to greet someone you know well.)\n\n\n\nLet me know if you\'d like to see more variations!\n'

# Prompt Templates

In [28]:

from langchain_core.prompts import ChatPromptTemplate

generic_template= "Translate the following into {language}:-"

prompt= ChatPromptTemplate.from_messages(
        [
            ("system", generic_template),
            ("user","{text}")
        ]
)

In [29]:
result= prompt.invoke({"language":"French", "text":"Hello How are you?"})

In [30]:
result.to_messages()

[SystemMessage(content='Translate the following into French:-', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello How are you?', additional_kwargs={}, response_metadata={})]

## Chaining together All Three Components with LCEL


In [31]:

chain=prompt|model|parser

chain.invoke({"language":"French", "text":"Hello How are you?"})

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with close friends)\n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n'